In [4]:
import os
import cv2
import matplotlib.pyplot as plt

from moviepy.editor import VideoFileClip
from IPython.display import HTML

from code.camera_cal import compute_camera_matrix
from code.image_pipeline import pipeline

%matplotlib notebook
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def sanity_check(left_curverad, right_curverad):
    if (left_curverad / right_curverad < 0.2) or (left_curverad / right_curverad > 5):
        return False
    else:
        return True

In [6]:
video_path = os.path.join(os.getcwd(), 'project_video.mp4')
cap = cv2.VideoCapture(video_path)
mtx, dist = compute_camera_matrix()

left_fit = None
right_fit = None

# fig = plt.figure()
# ax = fig.add_subplot(111)

output = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(output, fourcc, 20.0, (1280, 720))

font = cv2.FONT_HERSHEY_SIMPLEX
while cap.isOpened():
    ret, frame = cap.read()
    final_img, left_fit, right_fit, left_curverad, right_curverad, distance_to_center = pipeline(frame, mtx, dist, left_fit, right_fit)
    if sanity_check(left_curverad, right_curverad) is False:
        left_fit, right_fit = None, None
    if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
        break
    curverad = (left_curverad+right_curverad)/2
    string = 'curvature = {:5.2f}m, distance left of center = {:1.2f}m'.format(float(curverad), float(distance_to_center))
    cv2.putText(final_img, string, (10,50), font, 1, (255,255,255), 2)
    out.write(final_img) # Write out frame to video
    
out.release() 
cap.release()

error: C:\bld\opencv_1498163057615\work\opencv-3.2.0\modules\imgproc\src\undistort.cpp:193: error: (-215) dst.data != src.data in function cv::undistort
